In [2]:
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, classification_report, roc_auc_score, precision_recall_curve, auc

In [3]:
train_df = pd.read_csv("../../../dataset/open_world/openworld_train.csv")
test_df = pd.read_csv("../../../dataset/open_world/openworld_test.csv")

In [4]:
# feature/label 분리
target_col = train_df.columns[-1]   # 마지막 컬럼이 label

# open-world label 통합
train_df[target_col] = train_df[target_col].apply(lambda x: 0 if x == 95 else 1)
test_df[target_col]  = test_df[target_col].apply(lambda x: 0 if x == 95 else 1)

X_train = train_df.drop(columns=[target_col]).values
y_train = train_df[target_col].astype(np.float32).values
X_test  = test_df.drop(columns=[target_col]).values
y_test  = test_df[target_col].astype(np.float32).values

In [5]:
# Optuna Objective(K-Fold + Pruning)
def objective(trial):
    # Search Space
    hidden_layer_1 = trial.suggest_int("hidden_layer_1", 64, 512, step=32)
    hidden_layer_2 = trial.suggest_int("hidden_layer_2", 32, 256, step=16)
    alpha = trial.suggest_loguniform("alpha", 1e-5, 1e-2)
    learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-4, 5e-3)
    batch_size = trial.suggest_categorical("batch_size", [128, 256])
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    solver = trial.suggest_categorical("solver", ["adam", "sgd"])

    # K-Fold 설정
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []

    for fold_idx, (tr_idx, val_idx) in enumerate(kfold.split(X_train, y_train)):

        X_tr, X_val = X_train[tr_idx], X_train[val_idx]
        y_tr, y_val = y_train[tr_idx], y_train[val_idx]

        # MLP model
        mlp = MLPClassifier(
            hidden_layer_sizes=(hidden_layer_1, hidden_layer_2),
            activation=activation,
            solver=solver,
            alpha=alpha,
            batch_size=batch_size,
            learning_rate='adaptive',
            learning_rate_init=learning_rate_init,
            max_iter=300,
            early_stopping=True,
            validation_fraction=0.1,
            n_iter_no_change=10,
            random_state=42,
        )

        # train
        mlp.fit(X_tr, y_tr)

        # val
        pred = mlp.predict(X_val)
        f1_macro = f1_score(y_val, pred, average='macro')
        f1_scores.append(f1_macro)

        # Optuna pruning
        trial.report(f1_macro, fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return np.mean(f1_scores)

In [6]:
# Optuna Study
study = optuna.create_study(
    direction="maximize",
    sampler=TPESampler(seed=42),
    pruner=MedianPruner(n_warmup_steps=5)
)
study.optimize(objective, n_trials=100, show_progress_bar=True)

print("\n========== OPTUNA RESULTS ==========")
print(f"Best Macro-F1: {study.best_value:.4f}")
print("parameters:", study.best_params)

[I 2025-11-20 17:14:44,248] A new study created in memory with name: no-name-ccc00ab8-08d3-42ed-b0b4-d7a1dc03dadf


  0%|          | 0/100 [00:00<?, ?it/s]

/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_18644/2575538702.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-5, 1e-2)
/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_18644/2575538702.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-4, 5e-3)


[I 2025-11-20 17:17:29,400] Trial 0 finished with value: 0.7757940771314631 and parameters: {'hidden_layer_1': 224, 'hidden_layer_2': 256, 'alpha': 0.001570297088405539, 'learning_rate_init': 0.0010401663679887319, 'batch_size': 128, 'activation': 'tanh', 'solver': 'sgd'}. Best is trial 0 with value: 0.7757940771314631.


/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_18644/2575538702.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-5, 1e-2)
/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_18644/2575538702.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-4, 5e-3)


...

[I 2025-11-20 18:57:02,939] Trial 97 finished with value: 0.8045083205237871 and parameters: {'hidden_layer_1': 288, 'hidden_layer_2': 48, 'alpha': 0.0004370536498253204, 'learning_rate_init': 0.0011016374994181838, 'batch_size': 128, 'activation': 'relu', 'solver': 'sgd'}. Best is trial 14 with value: 0.8493686499044426.


/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_18644/2575538702.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-5, 1e-2)
/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_18644/2575538702.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-4, 5e-3)


[I 2025-11-20 18:57:57,718] Trial 98 finished with value: 0.8410435062957615 and parameters: {'hidden_layer_1': 256, 'hidden_layer_2': 32, 'alpha': 2.7438865239425338e-05, 'learning_rate_init': 0.0008891270396260732, 'batch_size': 128, 'activation': 'relu', 'solver': 'adam'}. Best is trial 14 with value: 0.8493686499044426.


/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_18644/2575538702.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform("alpha", 1e-5, 1e-2)
/var/folders/4q/3301trc50fz6_vl0qx1b71r00000gn/T/ipykernel_18644/2575538702.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-4, 5e-3)


[I 2025-11-20 18:58:50,126] Trial 99 finished with value: 0.8434997625002595 and parameters: {'hidden_layer_1': 320, 'hidden_layer_2': 80, 'alpha': 0.0003695408973446799, 'learning_rate_init': 0.004521425069641083, 'batch_size': 128, 'activation': 'relu', 'solver': 'adam'}. Best is trial 14 with value: 0.8493686499044426.

========== OPTUNA RESULTS ==========
Best Macro-F1: 0.8494
parameters: {'hidden_layer_1': 288, 'hidden_layer_2': 80, 'alpha': 3.813848083517318e-05, 'learning_rate_init': 0.0017053755428803074, 'batch_size': 128, 'activation': 'relu', 'solver': 'adam'}


In [7]:
best = study.best_params

mlp_best = MLPClassifier(
    hidden_layer_sizes=(best["hidden_layer_1"], best["hidden_layer_2"]),
    activation=best["activation"],
    solver=best["solver"],
    alpha=best["alpha"],
    batch_size=best["batch_size"],
    learning_rate="adaptive",
    learning_rate_init=best["learning_rate_init"],
    max_iter=300,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10,
    random_state=42
)

print("\n[INFO] Training best MLP model with full training data...")
mlp_best.fit(X_train, y_train)


[INFO] Training best MLP model with full training data...


MLPClassifier(alpha=3.813848083517318e-05, batch_size=128, early_stopping=True,
              hidden_layer_sizes=(288, 80), learning_rate='adaptive',
              learning_rate_init=0.0017053755428803074, max_iter=300,
              random_state=42)

In [9]:
test_pred = mlp_best.predict(X_test)
test_pred_proba = mlp_best.predict_proba(X_test)[:, 1]   # positive class 확률

test_acc = accuracy_score(y_test, test_pred)
test_f1_macro = f1_score(y_test, test_pred, average='macro')
test_f1_micro = f1_score(y_test, test_pred, average='micro')
test_f1_weighted = f1_score(y_test, test_pred, average='weighted')

# ROC-AUC (binary)
test_roc_auc = roc_auc_score(y_test, test_pred_proba)

# PR-AUC
prec, rec, _ = precision_recall_curve(y_test, test_pred_proba)
test_pr_auc = auc(rec, prec)

print("\n========== [TEST RESULTS] ==========")
print(f"Accuracy        : {test_acc:.4f}")
print(f"F1 (macro)      : {test_f1_macro:.4f}")
print(f"F1 (micro)      : {test_f1_micro:.4f}")
print(f"F1 (weighted)   : {test_f1_weighted:.4f}")
print(f"ROC-AUC         : {test_roc_auc:.4f}")
print(f"PR-AUC          : {test_pr_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, test_pred, digits=4))


========== [TEST RESULTS] ==========
Accuracy        : 0.8709
F1 (macro)      : 0.8538
F1 (micro)      : 0.8709
F1 (weighted)   : 0.8693
ROC-AUC         : 0.9358
PR-AUC          : 0.9625

Classification Report:
              precision    recall  f1-score   support

         0.0     0.8447    0.7667    0.8038      3000
         1.0     0.8829    0.9258    0.9038      5700

    accuracy                         0.8709      8700
   macro avg     0.8638    0.8462    0.8538      8700
weighted avg     0.8697    0.8709    0.8693      8700

